In [8]:
!pip install spacy folium geopy beautifulsoup4 requests
!python -m spacy download en_core_web_sm



  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import requests

# Odyssey (Samuel Butler translation) - from Project Gutenberg
homer_url = "https://www.gutenberg.org/files/1727/1727-0.txt"
homer_text = requests.get(homer_url).text

# Iliad (Samuel Butler translation) - from Project Gutenberg
iliad_url = "https://www.gutenberg.org/files/6130/6130-0.txt"
iliad_text = requests.get(iliad_url).text

In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp(homer_text)
locations = [ent.text for ent in doc.ents if ent.label_ in ("GPE", "LOC")]
unique_locations = list(set(locations))
print(f"Found {len(unique_locations)} unique locations")


Found 179 unique locations


In [19]:
sources = {
    "Odyssey": homer_text,
}

all_geo_data = []

for source, source_text in sources.items():
    doc = nlp(source_text)
    locs = list(set(ent.text for ent in doc.ents if ent.label_ in ("GPE", "LOC")))
    for place in locs:
        try:
            location = geolocator.geocode(place)
            if location:
                all_geo_data.append((place, location.latitude, location.longitude, source))
                sleep(1)
        except:
            continue


In [21]:
import folium


myth_map = folium.Map(location=[37.9838, 23.7275], zoom_start=5)

for name, lat, lon, source in all_geo_data:
    folium.Marker([lat, lon], popup=f"{name} ({source})").add_to(myth_map)

myth_map.save("myth_map_all_sources.html")
